In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

In [ ]:
url = 'https://comic.naver.com/webtoon/genre' 

resp = requests.get(url)
resp

<Response [200]>

In [ ]:
soup = BeautifulSoup(resp.content, 'lxml')
a_tags = soup.select('ul.spot a')
len(a_tags)

13

In [ ]:
g_urls, g_names = [], []
for a_tag in a_tags:
    g_urls.append('https://comic.naver.com'+a_tag['href'])
    g_names.append(a_tag.text)
g_urls = g_urls[3:]
g_names = g_names[3:]
g_urls, g_names

(['https://comic.naver.com/webtoon/genre?genre=daily',
  'https://comic.naver.com/webtoon/genre?genre=comic',
  'https://comic.naver.com/webtoon/genre?genre=fantasy',
  'https://comic.naver.com/webtoon/genre?genre=action',
  'https://comic.naver.com/webtoon/genre?genre=drama',
  'https://comic.naver.com/webtoon/genre?genre=pure',
  'https://comic.naver.com/webtoon/genre?genre=sensibility',
  'https://comic.naver.com/webtoon/genre?genre=thrill',
  'https://comic.naver.com/webtoon/genre?genre=historical',
  'https://comic.naver.com/webtoon/genre?genre=sports'],
 ['일상', '개그', '판타지', '액션', '드라마', '순정', '감성', '스릴러', '무협/사극', '스포츠'])

In [ ]:
titles, authors, ratings, genres = [], [], [], []

for i in tqdm(range(len(a_tags)-3)):
    genre, url = g_names[i], g_urls[i]
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')

    tit_tags = soup.select('ul.img_list li dl dt a')
    aut_tags = soup.select('ul.img_list dd.desc a')
    rat_tags = soup.select('div.rating_type strong')

    for tit_tag, aut_tag, rat_tag in zip(tit_tags, aut_tags, rat_tags):
        titles.append(tit_tag['title'])
        authors.append(aut_tag.text)
        ratings.append(rat_tag.text)
        genres.append(genre)

dic_toons = {
    "제목" : titles,
    "작가" : authors,
    "평점" : ratings,
    "장르" : genres
}

df_toons = pd.DataFrame(dic_toons)

100%|██████████| 10/10 [00:07<00:00,  1.34it/s]


In [ ]:
len(rat_tags)

23

In [ ]:
df_toons

,제목,작가,평점,장르
0,대학원 탈출일지,요다,9.97,일상
1,먹는 인생,홍끼,9.96,일상
2,독립일기,자까,9.97,일상
3,"안녕, 나의 수집",하린,9.95,일상
4,윌유메리미,마인드C,9.92,일상
...,...,...,...,...
2016,군주,박산하,8.50,스포츠
2017,미드나잇 체이서,석재윤,9.60,스포츠
2018,불릿 6미리,김도근,9.89,스포츠
2019,리턴,송래현,9.85,스포츠


In [ ]:
df_toons.to_csv('장르별 네이버 웹툰.csv', encoding='utf-16', index=False)

In [ ]:
df_toons.to_excel('장르별 네이버 웹툰.xlsx', index=False, encoding='utf-16')